In [ ]:
import pandas as pd

In [ ]:
dbcamhd = pd.read_json('dbcamhd.json', orient="records", lines=True)
dbcamhd.head()

In [ ]:
filenames = list(dbcamhd.filename[(dbcamhd.deployment == 2) & (dbcamhd.frame_count > 7500) & (dbcamhd.frame_count < 30000)])
filenames.sort()
filenames[0]

In [ ]:
frame_number = 7500

In [ ]:
import pycamhd as camhd
from skimage.transform import rescale

In [ ]:
def get_scaled_image(filename, frame_number, scale):
    frame = camhd.get_frame(filename, frame_number, 'rgb24')
    return np.uint8(rescale(frame, scale, mode='constant', anti_aliasing=True, preserve_range=True))

In [ ]:
import numpy as np
from dask import delayed
import dask.array as dsa

In [ ]:
delayed_frame_list = []
for i, filename in enumerate(filenames):
    delayed_frame = delayed(get_scaled_image)(filename, frame_number, 0.25)
    delayed_frame_list.append(dsa.from_delayed(delayed_frame, (3, 1080/4, 1920/4), np.uint8))
delayed_frame_array = dsa.stack(delayed_frame_list)
delayed_frame_array

#### Start a Dask cluster

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=1)
cluster

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client

In [ ]:
%%time
images = delayed_frame_array[0::20].compute()

In [ ]:
images.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rc('figure', figsize=(6,3))
image = images[1,:,:,:]
imgplot = plt.imshow(image)

In [ ]:
from numpngw import write_apng

In [ ]:
write_apng('funanimation.png', images, delay=100) #fun!